In [1]:
import sys
sys.path.insert(0, '../')

from interface import get_query_emb_batch, load_model, cosine_similarity_batch
from typing import Tuple, List, Dict, Union, Any
from colbert.modeling.checkpoint import Checkpoint

from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset



# pth_models = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/2801_models_Apple.csv"
pth_models = "/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_models_Apple.csv"
df_models = pd.read_csv(pth_models, sep=';')

pth_offers = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_Apple_triplets_offer_model_train.csv'
df_offers = pd.read_csv(pth_offers, sep=';')

pth_offers_test = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_test_Apple.csv'
df_offers_test = pd.read_csv(pth_offers, sep=';')

id_category = {
    # 3902: 'диктофоны, портативные рекордеры',
    # 510402: 'электронные книги',
    # 4302: 'автомобильные телевизоры, мониторы',
    # 2815: 'смарт-часы и браслеты',
    # 3901: 'портативные медиаплееры',
    # 3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    # 510401: 'планшетные компьютеры и мини-планшеты',
    # 2102: 'наушники, гарнитуры, наушники c микрофоном',
    # 3903: 'радиоприемники, радиобудильники, радиочасы',
    # 3907: 'магнитолы',
    # 280801: 'GPS-навигаторы'
    }

df_models = df_models[df_models['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers[df_offers['category_id'].isin(id_category.keys())].reset_index(drop=True)

df_offers

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,external_brand,external_type,true_match,false_match
0,Смартфоны Смартфон Apple iPhone 12 A2403 128ГБ...,7883241,1530924893,True,мобильные телефоны,2801,53060.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Смартфон Apple A2403 iPhone 12 черный, процесс...",Смартфоны,APPLE,Смартфоны,"Apple iPhone 12 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 128Gb, eSIM, Green"
1,Apple iPhone 14 Pro 128ГБ Silver (Серебристый)...,7883526,1626555801,True,мобильные телефоны,2801,118050.0,"[{'id': 14519, 'type': 'b', 'value': 1.0}, {'i...",Apple iPhone 14 Pro — супербыстрый смартфон с ...,Телефоны :: Телефоны Apple iPhone,Apple,NaN,"Apple iPhone 14 Pro 128Gb, Dual nano-SIM, Silver","Apple iPhone 14 Pro 128Gb, Dual nano-SIM, Gold"
2,Смартфон Apple iPhone 12 256Gb Blue (А2403),7883255,1690046919,False,мобильные телефоны,2801,77499.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple iPhone 12 256Gb Blue (А2403),Смартфоны и телефоны :: Apple :: iPhone 12,NaN,NaN,"Apple iPhone 12 256Gb, nano-Sim+eSIM, Синий","Apple iPhone 12 256Gb, nano-Sim+eSIM, Зеленый"
3,Смартфон Apple iPhone 12 128GB MGJA3HN/A Black,7883241,1540988497,True,мобильные телефоны,2801,62990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","iPhone 12. Во-первых, это быстро. iPhone 12 по...",Телефоны :: Смартфоны,Apple,Смартфон,"Apple iPhone 12 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 128Gb, eSIM, Green"
4,"Смартфон Apple iPhone 14 Pro 1 ТБ, Dual nano S...",7883536,1837804366,True,мобильные телефоны,2801,151170.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Новый волшебный способ взаимодействия с iPhone...,Все товары :: Электроника :: Телефоны :: Мобил...,Apple,Смартфон,"Apple iPhone 14 Pro 1Tb, Dual nano-SIM, Deep P...","Apple iPhone 14 Pro 1Tb, Dual eSIM, Space Black"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42715,Смартфон Apple iPhone 15 Plus 128GB eSIM Black...,7883661,1810174582,False,мобильные телефоны,2801,80490.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 15 Plus :: iPhone 15 Plus 1...,Apple,Смартфон,"Apple iPhone 15 Plus 128Gb, Dual eSIM, Black","Apple iPhone 15 512Gb, nano-Sim+eSIM, Черный"
42716,Смартфон Apple iPhone 14 Plus 128Gb Purple (фи...,7883483,1844276843,False,мобильные телефоны,2801,76090.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 1...,Apple,Смартфон,"Apple iPhone 14 Plus 128Gb, nano-Sim+eSIM, Фио...","Apple iPhone 14 Plus 128Gb, nano-Sim+eSIM, Тём..."
42717,Смартфон Apple iPhone 14 Plus 256Gb Purple (фи...,7883496,1844276848,False,мобильные телефоны,2801,88990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 2...,Apple,Смартфон,"Apple iPhone 14 Plus 256Gb, nano-Sim+eSIM, Purple","Apple iPhone 14 Plus 256Gb, Dual nano-SIM, Черный"
42718,Смартфон Apple iPhone 14 Plus 512Gb Blue (голу...,7883515,1851901283,False,мобильные телефоны,2801,93990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 5...,Apple,Смартфон,"Apple iPhone 14 Plus 512Gb, nano-Sim+eSIM, Blue","Apple iPhone 14 Plus 512Gb, nano-Sim+eSIM, (PR..."


In [2]:
# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# experiment = "colbert-5387"

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/2801_lr04_bsize_210_apple/none/2024-04/18/09.16.10/checkpoints/colbert-187-finish"
experiment = "colbert-187-finish"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"


In [3]:
checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

In [4]:
def get_query_emb(sentences: List[str], checkpoint: Checkpoint, batch_size: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.

    Returns:
        np.ndarray: An array of embeddings for the input sentences.
    """
    return checkpoint.queryFromText(sentences, bsize=batch_size).to("cpu").numpy()

def get_query_emb_batch(sentences: List[str], checkpoint: Checkpoint, batch_size: int, batch_size2: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences in batches using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.
        batch_size2 (int): The size of the sub-batches to split the input sentences into.

    Returns:
        np.ndarray: An array of embeddings for the input sentences. 
        Shape of the array is (len(sentences), 32, 768) for bert-base-multilingual-cased or (len(sentences), 32, 128) for colbertv2.0
    """
    embeddings_list = []
    
    for i in range(0, len(sentences), batch_size2):
        # print(f"batch: {min(i+batch_size2, len(sentences))}/{len(sentences)}")

        batch_sentences = sentences[i:i+batch_size2]
        embeddings = get_query_emb(batch_sentences, checkpoint, batch_size)
        embeddings_list.append(embeddings)

        torch.cuda.empty_cache()
    
    combined_embeddings = np.concatenate(embeddings_list, axis=0)
    return combined_embeddings

In [5]:

df_offers_shuffled = df_offers.sample(frac=1, random_state=42)

# Определяем размер тестовой выборки (например, 20%)
test_size = int(0.15 * len(df_offers_shuffled))

# Разделяем данные на тренировочную и тестовую выборки
df_train = df_offers_shuffled.iloc[:-test_size]
df_test = df_offers_shuffled.iloc[-test_size:]

# Пример: вывод размеров тренировочной и тестовой выборок
print("Размер тренировочной выборки:", len(df_train))
print("Размер тестовой выборки:", len(df_test))


Размер тренировочной выборки: 36312
Размер тестовой выборки: 6408


In [6]:
class MyDataset(Dataset):
    def __init__(self, offer_batch, true_match_batch, false_match_batch, checkpoint):
        self.offer_batch = offer_batch
        self.true_match_batch = true_match_batch
        self.false_match_batch = false_match_batch
        self.checkpoint = checkpoint

    def __len__(self):
        return len(self.offer_batch)

    def __getitem__(self, idx):
        offer_embs = get_query_emb_batch([self.offer_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)
        true_match_embs = get_query_emb_batch([self.true_match_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)
        false_match_embs = get_query_emb_batch([self.false_match_batch[idx]], self.checkpoint, batch_size=100, batch_size2=1000)

        y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
        y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])

        X_true = np.stack([offer_embs, true_match_embs], axis=1)
        X_false = np.stack([offer_embs, false_match_embs], axis=1)

        X = np.concatenate([X_true, X_false])
        y = np.concatenate([y_true, y_false])

        X_tensor = torch.tensor(X.reshape(X.shape[0], -1), dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32)  # изменено на float32

        # print(f"offer_embs: {np.shape(offer_embs)}")
        # print(f"true_match_embs: {np.shape(true_match_embs)}")
        # print(f"false_match_embs: {np.shape(false_match_embs)}")
        # print(f"y_true: {np.shape(y_true)}")
        # print(f"y_false: {np.shape(y_false)}")
        # print(f"X_true: {np.shape(X_true)}")
        # print(f"X_false: {np.shape(X_false)}")
        # print(f"X: {np.shape(X)}")
        # print(f"y: {np.shape(y)}")
        # print(f"X_tensor: {np.shape(X_tensor)}")
        # print(f"y_tensor: {np.shape(y_tensor)}")

        return X_tensor, y_tensor

# Определяем модель MLP
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2*32*768, 512) # На входе(batchX2, 2, 32, 768) - 32 токена, 768 размерность эмбеда каждого токена, 2=1оффер+1матч
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x


In [7]:
# Используем созданный класс для создания загрузчика данных
df = df_train.copy()
offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])
train_dataset = MyDataset(offer_batch, true_match_batch, false_match_batch, checkpoint)
train_loader = DataLoader(train_dataset, batch_size=10000, shuffle=True)

In [8]:
# Создаем экземпляр модели и оптимизатор
model = MLP()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
n_epochs = 1

# Обучаем модель
model.train()
for epoch in range(n_epochs):
    print(f"epoch: {epoch}")
    for inputs, labels in train_loader:
        # print(inputs, labels)
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 2*32*768))  # Преобразуем размерности входных данных
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()


epoch: 0


In [10]:
df = df_test.copy()

offer_batch = list(df['name'])
true_match_batch = list(df['true_match'])
false_match_batch = list(df['false_match'])

test_dataset = MyDataset(offer_batch, true_match_batch, false_match_batch, checkpoint)
test_loader = DataLoader(train_dataset, batch_size=10000, shuffle=True)

# Оцениваем точность модели на тестовом наборе
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.view(-1, 2*32*768))  # Преобразуем размерности входных данных
        predictions.extend(outputs.round().squeeze().tolist())
        true_labels.extend(labels.view(-1).tolist())
accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7969817140339281


In [ ]:
# epoch: 0
# Accuracy: 0.7969817140339281
# batch_size=100, batch_size=10000, 35 min

In [10]:
torch.save(model.state_dict(), 'model.pth')
model1 = MLP()
# Загружаем веса из файла
model1.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [11]:
model1

MLP(
  (fc1): Linear(in_features=49152, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [12]:
# Оцениваем точность модели на тестовом наборе
model1.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model1(inputs.view(-1, 2*32*768))  # Преобразуем размерности входных данных
        predictions.extend(outputs.round().squeeze().tolist())
        true_labels.extend(labels.view(-1).tolist())
accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.994767569949328


In [ ]:
"""
входные данные имеют размерность (2000, 2, 32, 768)
метки бинарной классификации имеют размерность (2000,)
Нужно сделать бинарный классификатор, принимающий на вход данные такой размерности, на базе multilayer perceptron
"""

# Предположим, что у вас есть данные и метки классов
# X имеет размерность (2000, 2, 32, 768)
# y имеет размерность (2000,)

# Разделяем данные на обучающий и тестовый наборы
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:

def batch_generator_from_df(df, batch_size):
    num_samples = len(df)
    num_batches = num_samples // batch_size
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        offer_batch = df['name'].iloc[start_idx:end_idx].values
        true_match_batch = df['true_match'].iloc[start_idx:end_idx].values
        false_match_batch = df['false_match'].iloc[start_idx:end_idx].values
        
        yield list(offer_batch), list(true_match_batch), list(false_match_batch)

def prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=1000, batch_size2=10000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=1000, batch_size2=10000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=1000, batch_size2=10000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])
    
    X_true = np.concatenate([offer_embs, true_match_embs], axis=1)
    X_false = np.concatenate([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    return X, y

def prepare_X_y_reshape(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=1000, batch_size2=10000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=1000, batch_size2=10000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=1000, batch_size2=10000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])
    
    # Создаем обучающий набор данных
    X_true = np.concatenate([offer_embs, true_match_embs], axis=1)
    X_false = np.concatenate([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    return X.reshape((X.shape[0], -1)), y

def prepare_X_y_stack(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=100, batch_size2=1000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=100, batch_size2=1000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=100, batch_size2=1000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])

    X_true = np.stack([offer_embs, true_match_embs], axis=1)
    X_false = np.stack([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    # indices = np.arange(len(X))
    # np.random.shuffle(indices)
    # X = X[indices]
    # y = y[indices]

    # print(f"offer_embs: {np.shape(offer_embs)}")
    # print(f"true_match_embs: {np.shape(true_match_embs)}")
    # print(f"false_match_embs: {np.shape(false_match_embs)}")
    # print(f"y_true: {np.shape(y_true)}")
    # print(f"y_false: {np.shape(y_false)}")
    # print(f"X_true: {np.shape(X_true)}")
    # print(f"X_false: {np.shape(X_false)}")
    # print(f"X: {np.shape(X)}")
    # print(f"y: {np.shape(y)}")

    return X, y


# df = df_test[:100].copy()

# offer_batch = list(df['name'])
# true_match_batch = list(df['true_match'])
# false_match_batch = list(df['false_match'])

# X_test, y_test = prepare_X_y_stack(offer_batch, true_match_batch, false_match_batch, checkpoint)

# print(np.shape(X_test))
# print(np.shape(y_test))


# # Преобразуем форму данных для PyTorch DataLoader
# X_test_flat = X_test.reshape(X_test.shape[0], -1)

# # Преобразуем данные в тензоры PyTorch
# X_test_tensor = torch.tensor(X_test_flat, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# # Создаем DataLoader для обучающего и тестового наборов
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# test_loader = DataLoader(test_dataset, batch_size=10)

offer_embs: (100, 32, 768)
true_match_embs: (100, 32, 768)
false_match_embs: (100, 32, 768)
y_true: (100,)
y_false: (100,)
X_true: (100, 2, 32, 768)
X_false: (100, 2, 32, 768)
X: (200, 2, 32, 768)
y: (200,)
(200, 2, 32, 768)
(200,)


In [ ]:

# # Оцениваем точность модели на тестовом наборе
# model.eval()
# predictions = []
# with torch.no_grad():
#     for inputs, _ in test_loader:
#         outputs = model(inputs)
#         predictions.extend(outputs.round().squeeze().tolist())

# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy:", accuracy)


Accuracy: 0.9775


In [ ]:
dict(df_train.iloc[0])

{'name': 'Apple iPhone 14 Pro 128GB серебристый',
 'model_id': 7883530,
 'offer_id': 1548811505,
 'modelized_by_barcode': False,
 'category_name': 'мобильные телефоны',
 'category_id': 2801,
 'price': 104990.0,
 'attrs': "[{'id': 9, 'type': 'r', 'value': 206.0}, {'id': 14519, 'type': 'b', 'value': 1.0}, {'id': 23613, 'type': 'e', 'value': 65430.0}, {'id': 25534, 'type': 'e', 'value': 716.0}, {'id': 25568, 'type': 'b', 'value': 1.0}, {'id': 25586, 'type': 'e', 'value': 733.0}, {'id': 25610, 'type': 'e', 'value': 787.0}]",
 'description': '100% Оригинальная техника! Гарантийное обслуживание от магазина - 1 год!!! // Бренд: Apple | Серия: iPhone 14 Pro | Операционная система: iOS 16 | Технология дисплея: OLED | Частота обновления: 120 Гц | Число пикселей на дюйм (PPI): 460 ppi | Контрастность: 2 000 000:1 | Яркость: 1600 нит | Тип процессора: Apple A16 Bionic | Встроенная память: 128 ГБ | Количество ядер процессора: 6-ядерный ЦП с 2 ядрами производительности и 4 ядрами эффективности | Раз

In [ ]:
offer_batch[0], true_match_batch[0], false_match_batch[0]

('Apple iPhone 14 Pro 128GB серебристый',
 'Apple iPhone 14 Pro 128Gb, nano-Sim+eSIM, Silver',
 'Apple iPhone 14 Pro 128Gb, nano-Sim+eSIM, Gold')